In [1]:
import ast
import pickle

from datasets import load_dataset, load_metric
from transformers import AutoConfig, Wav2Vec2Processor

In [2]:
data_files = {
    "train": "../../KEMDy20_v1_1/Splitting/Train.csv",
    "test": "../../KEMDy20_v1_1/Splitting/Test.csv"
}

In [3]:
dataset = load_dataset("csv", data_files = data_files)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

print(train_dataset)
print(test_dataset)

Found cached dataset csv (C:/Users/Yechani/.cache/huggingface/datasets/csv/default-abf3189d151d4659/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['Segment ID', 'path', 'emotion_Prob', 'emotion_Multi', 'emotion_Eval01F', 'emotion_Eval02M', 'emotion_Eval03M', 'emotion_Eval04F', 'emotion_Eval05M', 'emotion_Eval06F', 'emotion_Eval07F', 'emotion_Eval08M', 'emotion_Eval09F', 'emotion_Eval10M'],
    num_rows: 10897
})
Dataset({
    features: ['Segment ID', 'path', 'emotion_Prob', 'emotion_Multi', 'emotion_Eval01F', 'emotion_Eval02M', 'emotion_Eval03M', 'emotion_Eval04F', 'emotion_Eval05M', 'emotion_Eval06F', 'emotion_Eval07F', 'emotion_Eval08M', 'emotion_Eval09F', 'emotion_Eval10M'],
    num_rows: 2565
})


In [4]:
import torchaudio
import librosa
import IPython.display as ipd
import numpy as np

idx = np.random.randint(0, len(train_dataset))
sample = train_dataset[idx]
path = sample["path"]
label = sample["emotion_Prob"]


print(f"ID Location: {idx}")
print(f"      Label: {label}")
print()

speech, sr = torchaudio.load(path)
speech = speech[0].numpy().squeeze()
speech = librosa.resample(y=np.asarray(speech), orig_sr=sr, target_sr=16000)
ipd.Audio(data=np.asarray(speech), autoplay=True, rate=16000)

ID Location: 614
      Label: [0.0, 0.0, 0.0, 0.1, 0.9, 0.0, 0.0]



In [5]:
input_column = "path"
output_column = "emotion_Prob"

In [6]:
model_name = "jungjongho/wav2vec2-xlsr-korean-speech-emotion-recognition3"
pooling_mode = "mean"

In [7]:
with open("../../KEMDy20_v1_1/Splitting/Label.txt", "rb") as f:
    label_list = pickle.load(f)

In [8]:
num_labels = len(label_list)

In [9]:
config = AutoConfig.from_pretrained(
    model_name,
    num_labels = num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf"
)

setattr(config, "pooling_mode", pooling_mode)

In [10]:
processor = Wav2Vec2Processor.from_pretrained(model_name)
target_sampling_rate = processor.feature_extractor.sampling_rate
print(f"The target sampling rate: {target_sampling_rate}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


The target sampling rate: 16000


In [11]:
def speech_file_to_array_fn(path):
    speech_array, sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech

def preprocess_function(examples):
    speech_list = [speech_file_to_array_fn(path) for path in examples[input_column]]
    target_list = [ast.literal_eval(label) for label in examples[output_column]]

    result = processor(speech_list, sampling_rate=target_sampling_rate)
    result["labels"] = target_list

    return result

In [12]:
train_dataset = train_dataset.map(
    preprocess_function,
    batch_size=100,
    batched=True
)

test_dataset = test_dataset.map(
    preprocess_function,
    batch_size=100,
    batched=True
)

Loading cached processed dataset at C:\Users\Yechani\.cache\huggingface\datasets\csv\default-abf3189d151d4659\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-dadc9f313659a6cf.arrow
Loading cached processed dataset at C:\Users\Yechani\.cache\huggingface\datasets\csv\default-abf3189d151d4659\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-4b396ec641cdb23b.arrow


In [13]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput

@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

In [14]:
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)

class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x


class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        logits = self.classifier(hidden_states)
        logits = torch.softmax(logits, dim=-1)
        
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.num_labels), labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [15]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import torch

import transformers
from transformers import Wav2Vec2Processor

@dataclass
class DataCollatorCTCWithPadding:

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["labels"] for feature in features]

        d_type = torch.long if isinstance(label_features[0], int) else torch.float

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch["labels"] = torch.tensor(label_features, dtype=d_type)

        return batch

In [16]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [17]:
import numpy as np
from transformers import EvalPrediction
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics(p: EvalPrediction):
    y_pred = p.predictions.astype(np.float32)
    y_true = p.label_ids.astype(np.float32)
    
    precision, recall, f1_score, support = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    
    return {"weighted_f1": f1_score}

In [18]:
model = Wav2Vec2ForSpeechClassification.from_pretrained(
    model_name,
    config=config,
    ignore_mismatched_sizes=True
)

Some weights of Wav2Vec2ForSpeechClassification were not initialized from the model checkpoint at jungjongho/wav2vec2-xlsr-korean-speech-emotion-recognition3 and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([6, 1024]) in the checkpoint and torch.Size([7, 1024]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([6]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
model.freeze_feature_extractor()

In [20]:
import os
from transformers import TrainingArguments

output_path = "../../KEMDy20_v1_1/content/"
if not os.path.exists(output_path):
    os.mkdir(output_path)

training_args = TrainingArguments(
    output_dir=output_path,
    logging_dir="../../KEMDy20_v1_1/log/",
    learning_rate=2e-05,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    fp16=True,
    save_steps= 50,
    save_total_limit=2
)

In [21]:
from typing import Any, Dict, Union

import torch
from packaging import version
from torch import nn

from transformers import (
    Trainer,
    is_apex_available,
)

if is_apex_available():
    from apex import amp

if version.parse(torch.__version__) >= version.parse("1.6"):
    _is_native_amp_available = True
    from torch.cuda.amp import autocast

class CTCTrainer(Trainer):
    def __init__(
        self,
        use_amp: Optional[bool] = None,
        *args, **kwargs
    ):
        super().__init__(*args, **kwargs)
        self.use_amp = self.args.fp16 if use_amp is None else use_amp

    def training_step(self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:

        model.train()
        inputs = self._prepare_inputs(inputs)

        if self.use_amp:
            with autocast():
                loss = self.compute_loss(model, inputs)

        if self.args.gradient_accumulation_steps > 1:
            loss = loss / self.args.gradient_accumulation_steps

        if self.use_amp:
            self.scaler.scale(loss).backward()

        return loss.detach()

In [22]:
trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
    use_amp=True
)

In [23]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [24]:
trainer.train()

C:\Users\Yechani\.conda\envs\ETRI\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,1.460000
1000,1.434900
1500,1.436100
2000,1.445300
2500,1.435900


TrainOutput(global_step=2724, training_loss=1.4423529335055583, metrics={'train_runtime': 3051.757, 'train_samples_per_second': 3.571, 'train_steps_per_second': 0.893, 'total_flos': 2.6821294152154276e+18, 'train_loss': 1.4423529335055583, 'epoch': 1.0})